hier einmal die hoffentlich schöne Version des Notebooks, welches es erlaubt alle Anläufe von Kreuzfahrtschiffen in Kiel herunterzuladen. 
Quelle:https://kreuzfahrtschiffehamburg.de/hafen-kiel/

In [53]:
import pandas as pd
import numpy as np  
print(np.__version__)
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup

2.1.1


In [54]:
#define function to get the data from the website
def get_data(year):
    url = f'https://kreuzfahrtschiffehamburg.de/kiel-{year}/'
    response = requests.get(url)
    html_content = response.text
    #parse the html content
    soup = BeautifulSoup(html_content, "lxml")
    #text = soup.get_text()
    # Extract text from all paragraphs
    paragraphs = soup.find_all('p')
    paragraphs = [p.get_text() for p in paragraphs]
    #print(paragraphs)


    #I want to remove \xa0 \xa from the text

    removed =[]
    for p in paragraphs:
        text = p.replace('\xa0', ' ')
        #text = text.strip()
        #text = text.split('\n')
        removed.append(text)

    #print(removed)

    #remove first 7 elements
    removed = removed[2:]
    removed = removed[:-4]

    #remove empty strings and replace them with Nan
    for entry in removed:
        if entry == '':
            entry = np.nan
    #print(removed)

    return removed

In [55]:
#define function to clean the data
def split_list_by_word(input_list, split_word):
    """
    Splits a list into sublists each time the split_word is encountered.

    Parameters:
    input_list (list): The list to be split.
    split_word (str): The word to split the list by.

    Returns:
    list: A list of sublists.
    """
    sublists = []
    temp_list = []

    for entry in input_list:
        if split_word in entry:
            if temp_list:
                sublists.append(temp_list)
                temp_list = []
        temp_list.append(entry)

    if temp_list:
        sublists.append(temp_list)

    
    print(sublists)

    return sublists



In [56]:
#delete the first 5 elements of each sublist
def remove_first_entries(sublists):
    sublists_cleaned = []
    for list in sublists:
        list = list[5:]
        sublists_cleaned.append(list)

    print(sublists_cleaned)
    return sublists_cleaned

In [57]:
#one some websites there were empty strings which have to be deleted
            
def remove_empty_strings(sublists_cleaned):
    for sublist in sublists_cleaned:
        sublist[:] = [entry for entry in sublist if entry != ' ']
    
    print(sublists_cleaned)
    return sublists_cleaned

In [58]:
#define function to get numbers of entries in each sublist
def get_numbers(sublists_cleaned):
    rows = []
    for sublist in sublists_cleaned:
        #get the indices of the days of the week
        indices = [i for i, x in enumerate(sublist) if x in ['Mo', 'Di', 'Mi', 'Do', 'Fr', 'Sa', 'So']]
        if not indices:
            continue  # skip sublist if no days of the week are found
        #print(indices)
        #add up the indices to get the number of rows in each dataframe
        numbers = len(indices)
        rows.append(numbers)
        
    print(rows)
    return rows
    #print(rows)

In [59]:
#define function to create the df and concatenate the data
def create_df(sublists_cleaned, rows):
    dataframes = []
    for list,number in zip(sublists_cleaned, rows):
        print(list)
        print(number)
        #split list in chunks
        chunks = [list[i:i + number] for i in range(0, len(list), number)]
        df = pd.DataFrame(chunks).transpose()
        # Rename columns for clarity
        df.columns = ['Weekday','Arrival_date', 'Departure_date','Ship','Pier']
        dataframes.append(df)

    # Concatenate the dataframes
    df_complete = pd.concat(dataframes, ignore_index=True)

    #reshape the arrival and departure dates
    df_final = df_complete
    print(type(df_complete['Arrival_date']))
    df_final['Arrival_date'] = df_complete['Arrival_date'].str.split(' ').str[0]
    df_final['Departure_date'] = df_complete['Departure_date'].str.split(' ').str[0]
    print(df_final)
    return df_complete


In [60]:
#define function to save the df to a csv file
def save_df(df, year):
    df.to_csv(f'kiel_{year}.csv', index=False)

Nun habe ich alle funktionen erstellt. Es sollte nun möglich sein für jedes Jahr die Daten runter zu laden und als csv Dateien abzuspeichern

In [61]:
#lets try and get the data for 2012
year = 2012
removed = get_data(year)
sublists = split_list_by_word(removed, 'Tag')
sublists_cleaned = remove_first_entries(sublists)
rows = get_numbers(sublists_cleaned)
df_2012 = create_df(sublists_cleaned, rows)
save_df(df_2012, year)




[['Tag', 'Ankunft   Uhrzeit', 'Abfahrt   Uhrzeit', 'Schiff', 'Liegeplatz', 'So', '31.03.12  08:00', '31.03.12  17:00', 'AIDAcara', 'Ostseekai'], ['Tag', 'Ankunft   Uhrzeit', 'Abfahrt   Uhrzeit', 'Schiff', 'Liegeplatz', 'So', 'So', 'So', 'So', 'So', '07.04.12  08:00', '14.04.12  08:00', '21.04.12  08:00', '28.04.12  08:00', '28.04.12  08:00', '07.04.12  17:00', '14.04.12  17:00', '21.04.12  17:00', '28.04.12  17:00', '28.04.12  16:00', 'AIDAcara', 'AIDAcara', 'AIDAcara', 'AIDAcara', 'MSC Lirica', 'Ostseekai', 'Ostseekai', 'Ostseekai', 'Ostseekai', 'Ostseekai'], ['Tag', 'Ankunft   Uhrzeit', 'Abfahrt   Uhrzeit', 'Schiff', 'Liegeplatz', 'Mi', 'Do', 'Sa', 'Sa', 'So', 'Mo', 'Mo', 'Do', 'Sa', 'Sa', 'Mo', 'Di', 'Di', 'Do', 'Sa', 'Sa', 'Sa', 'So', 'Di', 'Mi', 'Do', 'Fr', 'Sa', 'Sa', 'Sa', 'Mi', '02.05.12  08:00', '03.05.12  09:00', '05.05.12  08:00', '05.05.12  10:00', '06.05.12  09:00', '07.05.12  07:00', '07.05.12  09:00', '10.05.12  10:00', '12.05.12  08:00', '12.05.12  10:00', '14.05.12  08

In [62]:
#amd now for 2013
year = 2013
removed = get_data(year)
sublists = split_list_by_word(removed, 'Tag')
sublists_cleaned = remove_first_entries(sublists)
rows = get_numbers(sublists_cleaned)
df_2013 = create_df(sublists_cleaned, rows)
save_df(df_2013, year)

[['Tag', 'Ankunft   Uhrzeit', 'Abfahrt   Uhrzeit', 'Schiff', 'Liegeplatz', 'Sa', 'Mi', 'Sa', 'Di', 'Mi', 'Do', 'Sa', 'So', 'Di', '13.04.13  08:00', '17.04.13  08:00', '20.04.13  08:00', '23.04.13  07:30', '24.04.13  08:00', '25.04.13  10:00', '27.04.13  08:00', '28.04.13  10:00', '30.04.13  08:00', '13.04.13  17:00', '17.04.13  17:00', '20.04.13  17:00', '23.04.13  17:30', '24.04.13  17:00', '25.04.13  16:00', '27.04.13  17:00', '28.04.13  16:00', '30.04.13  23:00', 'AIDAcara', 'AIDAcara', 'AIDAcara', 'Astor', 'AIDAcara', 'MSC Musica', 'AIDAcara', 'MSC Musica', 'Rotterdam', 'Ostseekai', 'Ostseekai', 'Ostseekai', 'Norwegenkai', 'Ostseekai', 'Ostseekai', 'Ostseekai', 'Ostseekai', 'Ostseekai'], ['Tag', 'Ankunft   Uhrzeit', 'Abfahrt   Uhrzeit', 'Schiff', 'Liegeplatz', 'Mi', 'Mi', 'Sa', 'So', 'So', 'Di', 'Sa', 'Sa', 'So', 'So', 'Mo', 'Do', 'Sa', 'Sa', 'So', 'Do', 'Sa', 'Sa', 'So', 'Di', 'Mi', 'Mi', 'Do', 'Fr', '01.05.13  08:00', '01.05.13  09:00', '04.05.13  08:00', '05.05.13  08:00', '05.0

In [63]:
#and now for 2014
year = 2014
removed = get_data(year)
sublists = split_list_by_word(removed, 'Tag')
sublists_cleaned = remove_first_entries(sublists)
rows = get_numbers(sublists_cleaned)
df_2014 = create_df(sublists_cleaned, rows)
save_df(df_2014, year)


[['Tag', 'Ankunft   Uhrzeit', 'Abfahrt   Uhrzeit', 'Schiff', 'Liegeplatz', 'Sa', 'Do', 'Sa', 'Sa', 'Sa', 'Mi', '05.04.14  08:00', '10.04.14  08:00', '12.04.14  08:00', '19.04.14  08:00', '26.04.14  08:00', '30.04.14  09:00', '05.04.14  17:00', '10.04.14  17:00', '12.04.14  17:00', '19.04.14  17:00', '26.04.14  17:00', '30.04.14  17:00', 'AIDAcara', 'AIDAcara', 'AIDAcara', 'AIDAcara', 'AIDAcara', 'Costa Pacifica', 'Ostseekai', 'Ostseekai', 'Ostseekai', 'Ostseekai', 'Ostseekai', 'Ostseekai'], ['Tag', 'Ankunft   Uhrzeit', 'Abfahrt   Uhrzeit', 'Schiff', 'Liegeplatz', 'Sa', 'Sa', 'Sa', 'Sa', 'So', 'Mi', 'Do', 'Sa', 'Sa', 'Mo', 'Mo', 'Mi', 'Do', 'Sa', 'Sa', 'So', 'Di', 'Do', 'Sa', 'Sa', 'Sa', '03.05.14  08:00', '10.05.14  08:00', '10.05.14  08:00', '10.05.14  10:00', '11.05.14  09:00', '14.05.14  10:00', '15.05.14  08.00', '17.05.14  08:00', '17.05.14  10:00', '19.05.14  07:00', '19.05.14  08:00', '21.05.14  07:00', '22.05.14  09:00', '24.05.14  08:00', '24.05.14  10:00', '25.05.14  09:00', 

In [64]:
#and now for 2015
year = 2015
removed = get_data(year)
sublists = split_list_by_word(removed, 'Tag')
sublists_cleaned = remove_first_entries(sublists)
sublists_cleaned = remove_empty_strings(sublists_cleaned)
rows = get_numbers(sublists_cleaned)
df_2015 = create_df(sublists_cleaned, rows)
save_df(df_2015, year)




[['Tag', 'Ankunft   Uhrzeit', 'Abfahrt   Uhrzeit', 'Schiff', 'Liegeplatz', 'Sa', 'Sa', 'Sa', 'Mo', 'Fr', 'Sa', 'Di', 'Di', '04.04.15  08:00', '11.04.15  08:00', '18.04.15  08:00', '20.04.15  08:00', '24.04.15  08:00', '25.04.15  08:00', '28.04.15  08:00', '28.04.15  08:00', '04.04.15  17:00', '11.04.15  17:00', '18.04.15  17:00', '20.04.15  18:00', '24.04.15  18:00', '25.04.15  17:00', '28.04.15  17:00', '28.04.15  18:00', 'AIDAcara', 'AIDAcara', 'AIDAcara', 'AIDAcara', 'AIDAluna', 'AIDAcara', 'Costa Pacifica', 'AIDAluna', 'Ostseekai', 'Ostseekai', 'Ostseekai', 'Ostseekai', 'Ostseekai', 'Ostseekai', 'Ostseekai', 'Ostseekai'], ['Tag', 'Ankunft   Uhrzeit', 'Abfahrt   Uhrzeit', 'Schiff', 'Liegeplatz', 'Sa', 'Sa', 'Sa', 'Sa', 'Sa', 'Di', 'Do', 'Do', 'Sa', 'Sa', 'Sa', 'So', 'So', 'Mi', 'Mi', 'Do', 'Do', 'Sa', 'Sa', 'Mo', 'Sa', 'Sa', 'Sa', 'So', '02.05.15  08:00', '02.05.15  08:00', '09.05.15  07:00', '09.05.15  08:00', '09.05.15  08:00', '12.05.15  08:00', '14.05.15  07:00', '14.05.15  07:0

In [65]:
#and now for 2016
year = 2016
removed = get_data(year)
sublists = split_list_by_word(removed, 'Tag')
sublists_cleaned = remove_first_entries(sublists)
sublists_cleaned = remove_empty_strings(sublists_cleaned)
rows = get_numbers(sublists_cleaned)
df_2016 = create_df(sublists_cleaned, rows)
save_df(df_2016, year)

[['Tag', 'Ankunft   Uhrzeit', 'Abfahrt   Uhrzeit', 'Schiff', 'Liegeplatz', 'Mo', 'Fr', 'Sa', '25.04.16  08:00', '29.04.16  08:00', '30.04.16  10:00', '25.04.16  18:00', '29.04.16  18:00', '30.04.16  18:00', 'AIDAluna', 'AIDAluna', 'MSC Musica', 'Ostseekai', 'Ostseekai', 'Ostuferhafen'], ['Tag', 'Ankunft   Uhrzeit', 'Abfahrt   Uhrzeit', 'Schiff', 'Liegeplatz', 'Di', 'Sa', 'Sa', 'Sa', 'Mi', 'Sa', 'Sa', 'So', 'So', 'Do', 'Sa', 'Sa', 'Sa', 'So', 'So', 'Mo', 'Sa', 'So', 'So', 'Mo', 'Di', '03.05.16  08:00', '07.05.16  08:00', '07.05.16  08:00', '07.05.16  08:00', '11.05.16  08:00', '14.05.16  07:00', '14.05.16  10:00', '15.05.16  08:00', '15.05.16  08:00', '19.05.16  08:00', '21.05.16  06:30', '21.05.16  08:00', '21.05.16  08:00', '22.05.16  07:00', '22.05.16  08:00', '23.05.16  08:00', '28.05.16  10:00', '29.05.16  06:30', '29.05.16  08:00', '30.05.16  21:30', '31.05.16  08:00', '03.05.16  18:00', '07.05.16  18:00', '07.05.16  18:00', '07.05.16  18:00', '11.05.16  18:00', '14.05.16  17:00',

In [66]:
#and now for 2017
year = 2017
removed = get_data(year)
sublists = split_list_by_word(removed, 'Tag')
sublists_cleaned = remove_first_entries(sublists)
rows = get_numbers(sublists_cleaned)
df_2017 = create_df(sublists_cleaned, rows)
save_df(df_2017, year)

[['Tag', 'Ankunft   Uhrzeit', 'Abfahrt   Uhrzeit', 'Schiff', 'Liegeplatz', 'So', 'So', 'So', 'Di', 'Sa', 'So', '09.04.17  08:00', '16.04.17  08:00', '23.04.17  08:00', '25.04.17  02:00', '29.04.17  10:00', '30.04.17  08:00', '09.04.17  17:00', '16.04.17  17:00', '23.04.17  17:00', '25.04.17  17:00', '29.04.17  18:00', '30.04.17  17:00', 'AIDAcara', 'AIDAcara', 'AIDAcara', 'Astor', 'MSC Fantasia', 'AIDAcara', 'Ostseekai', 'Ostseekai', 'Ostseekai', 'Norwegenkai', 'Ostseekai', 'Ostseekai'], ['Tag', 'Ankunft   Uhrzeit', 'Abfahrt   Uhrzeit', 'Schiff', 'Liegeplatz', 'Sa', 'Sa', 'So', 'Do', 'Fr', 'Sa', 'Di', 'Mi', 'Do', 'Fr', 'Fr', 'Sa', 'So', 'Mo', 'Di', 'Di', 'Fr', 'Sa', 'Sa', 'Sa', 'So', 'Di', 'Mi', '06.05.17  10:00', '06.05.17  11:00', '07.05.17  08:00', '11.05.17  06:30', '12.05.17  05:30', '13.05.17  10:00', '16.05.17  08:00', '17.05.17  10:00', '18.05.17  08:00', '19.05.17  06:30', '19.05.17  06:30', '20.05.17  10:00', '21.05.17  08:00', '22.05.17  08:00', '23.05.17  06:30', '23.05.17 

In [67]:
#and now for 2018
year = 2018
removed = get_data(year)
sublists = split_list_by_word(removed, 'Tag')
sublists_cleaned = remove_first_entries(sublists)
rows = get_numbers(sublists_cleaned)
df_2018 = create_df(sublists_cleaned, rows)
save_df(df_2018, year)

[['Tag', 'Ankunft   Uhrzeit', 'Abfahrt   Uhrzeit', 'Schiff', 'Liegeplatz', 'Fr', 'So', 'So', 'So', 'Do', 'Fr', 'Sa', 'Sa', 'So', 'So', '06.04.18  11:30', '08.04.18  08:00', '15.04.18  08:00', '22.04.18  08:00', '26.04.18  08:00', '27.04.18  06:30', '28.04.18  08:00', '28.04.18  10:00', '29.04.18  06:30', '29.04.18  08:00', '06.04.18  23:00', '08.04.18  18:00', '15.04.18  18:00', '22.04.18  18:00', '26.04.18  18:00', '27.04.18  23:00', '28.04.18  18:00', '28.04.18  18:00', '29.04.18  19:00', '29.04.18  18:00', 'Boudicca', 'AIDAcara', 'AIDAcara', 'AIDAcara', 'AIDAluna', 'Mein Schiff 1', 'AIDAluna', 'MSC Preziosa', 'Mein Schiff 1', 'AIDAcara', 'Ostseekai', 'Ostseekai', 'Ostseekai', 'Ostseekai', 'Ostseekai', 'Ostseekai', 'Ostseekai', 'Ostseekai', 'Ostseekai', 'Ostseekai'], ['Tag', 'Ankunft   Uhrzeit', 'Abfahrt   Uhrzeit', 'Schiff', 'Liegeplatz', 'Fr', 'Fr', 'Sa', 'Sa', 'Mi', 'Fr', 'Sa', 'Sa', 'So', 'So', 'Di', 'Do', 'Do', 'Sa', 'Sa', 'Sa', 'Sa', 'So', 'Mo', 'Mo', 'Mo', 'Mi', 'Fr', 'Sa', 'S

In [68]:
#and now for 2019
year = 2019
removed = get_data(year)
sublists = split_list_by_word(removed, 'Tag')
sublists_cleaned = remove_first_entries(sublists)
rows = get_numbers(sublists_cleaned)
df_2019 = create_df(sublists_cleaned, rows)
save_df(df_2019, year)

[['Tag', 'Ankunft   Uhrzeit', 'Abfahrt   Uhrzeit', 'Schiff', 'Liegeplatz', 'Sa', '05.01.19  09:30', '23.01.19  19:00', 'Mein Schiff 2', 'Ostseekai'], ['Tag', 'Ankunft   Uhrzeit', 'Abfahrt   Uhrzeit', 'Schiff', 'Liegeplatz', 'So', 'Fr', 'So', 'Sa', 'So', 'Mi', 'Sa', 'So', '07.04.19  08:00', '12.04.19  08:00', '14.04.19  08:00', '20.04.19  08:00', '21.04.19  08:00', '24.04.19  06:00', '27.04.19  07:00', '28.04.19  08:00', '07.04.19  17:00', '12.04.19  17:00', '14.04.19  18:00', '20.04.19  18:00', '21.04.19  18:00', '24.04.19  17:00', '27.04.19  19:00', '28.04.19  18:00', 'AIDAcara', 'Viking Sky', 'AIDAcara', 'Viking Sky', 'AIDAcara', 'Astor', 'MSC Meraviglia', 'AIDAcara', 'Ostseekai', 'Ostseekai', 'Ostseekai', 'Ostseekai', 'Ostseekai', 'Ostseekai', 'Ostuferhafen', 'Ostseekai'], ['Tag', 'Ankunft   Uhrzeit', 'Abfahrt   Uhrzeit', 'Schiff', 'Liegeplatz', 'Sa', 'Do', 'Fr', 'Fr', 'Sa', 'So', 'Mo', 'Di', 'Di', 'Do', 'Do', 'Fr', 'Sa', 'So', 'Mo', 'Mo', 'Mi', 'Do', 'Do', 'Do', 'Sa', 'So', 'Mo', '

In [ ]:
#create one big df
df_all = pd.concat([df_2012, df_2013, df_2014, df_2017, df_2018, df_2019], ignore_index=True)
print(df_all)


    Weekday Arrival_date Departure_date       Ship         Pier
0        So     31.03.12       31.03.12   AIDAcara    Ostseekai
1        So     07.04.12       07.04.12   AIDAcara    Ostseekai
2        So     14.04.12       14.04.12   AIDAcara    Ostseekai
3        So     21.04.12       21.04.12   AIDAcara    Ostseekai
4        So     28.04.12       28.04.12   AIDAcara    Ostseekai
..      ...          ...            ...        ...          ...
831      So     29.09.19       29.09.19  AIDAbella    Ostseekai
832      Do     03.10.19       03.10.19  AIDAbella    Ostseekai
833      Sa     05.10.19       05.10.19  AIDAprima    Ostseekai
834      Di     08.10.19       08.10.19      Astor  Norwegenkai
835      Sa     12.10.19       12.10.19  AIDAprima    Ostseekai

[836 rows x 5 columns]


In [71]:
#get subset of the df with relevant data
df_subset = df_all[['Arrival_date', 'Ship']]

#reshape the arival date to YYYY-MM-DD
df_subset['Arrival_date'] = pd.to_datetime(df_subset['Arrival_date'])
print(df_subset)

    Arrival_date       Ship
0     2012-03-31   AIDAcara
1     2012-07-04   AIDAcara
2     2012-04-14   AIDAcara
3     2012-04-21   AIDAcara
4     2012-04-28   AIDAcara
..           ...        ...
831   2019-09-29  AIDAbella
832   2019-03-10  AIDAbella
833   2019-05-10  AIDAprima
834   2019-08-10      Astor
835   2019-12-10  AIDAprima

[836 rows x 2 columns]


/tmp/ipykernel_3602/571021068.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_subset['Arrival_date'] = pd.to_datetime(df_subset['Arrival_date'])
/tmp/ipykernel_3602/571021068.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['Arrival_date'] = pd.to_datetime(df_subset['Arrival_date'])


In [76]:
#some rows have the same date, so I want to group them by date and count the number of ships
df_grouped = df_subset.groupby('Arrival_date').count()
#reset the index
df_grouped = df_grouped.reset_index()
df_grouped.columns = ['Arival_date','Number_of_ships']
df_grouped['Ship'] = True
print(df_grouped)

    Arival_date  Number_of_ships  Ship
0    2012-01-07                2  True
1    2012-01-09                2  True
2    2012-02-05                1  True
3    2012-02-06                2  True
4    2012-02-07                1  True
..          ...              ...   ...
575  2019-11-09                1  True
576  2019-12-04                1  True
577  2019-12-07                1  True
578  2019-12-09                1  True
579  2019-12-10                1  True

[580 rows x 3 columns]


In [77]:
#export the df to a csv file
df_grouped.to_csv('Alle_Schiffe.csv', index=False)